# Hide

In [6]:
text = """
Overview of LangChain v0.2
What’s new in LangChain?

The following features have been added during the development of 0.1.x:

    Better streaming support via the Event Streaming API.
    Standardized tool calling support
    A standardized interface for structuring output
    @chain decorator to more easily create RunnableLambdas
    https://python.langchain.com/docs/expression_language/how_to/inspect/
    In Python, better async support for many core abstractions (thank you @cbornet!!)
    Include response metadata in AIMessage to make it easy to access raw output from the underlying models
    Tooling to visualize your runnables or your langgraph app
    Interoperability of chat message histories across most providers
    Over 20+ partner packages in python for popular integrations

What’s coming to LangChain?

    We’ve been working hard on langgraph. We will be building more capabilities on top of it and focusing on making it the go-to framework for agent architectures.
    Vectorstores V2! We’ll be revisiting our vectorstores abstractions to help improve usability and reliability.
    Better documentation and versioned docs!
    We’re planning a breaking release (0.3.0) sometime between July-September to upgrade to full support of Pydantic 2, and will drop support for Pydantic 1 (including objects originating from the v1 namespace of Pydantic 2).

What changed?

Due to the rapidly evolving field, LangChain has also evolved rapidly.

This document serves to outline at a high level what has changed and why.
TLDR

As of 0.2.0:

    This release completes the work that we started with release 0.1.0 by removing the dependency of langchain on langchain-community.
    langchain package no longer requires langchain-community . Instead langchain-community will now depend on langchain-core and langchain .
    User code that still relies on deprecated imports from langchain will continue to work as long langchain_community is installed. These imports will start raising errors in release 0.4.x.

As of 0.1.0:

    langchain was split into the following component packages: langchain-core, langchain, langchain-community, langchain-[partner] to improve the usability of langchain code in production settings. You can read more about it on our blog.

Ecosystem organization

By the release of 0.1.0, LangChain had grown to a large ecosystem with many integrations and a large community.

To improve the usability of LangChain in production, we split the single langchain package into multiple packages. This allowed us to create a good foundation architecture for the LangChain ecosystem and improve the usability of langchain in production.

Here is the high level break down of the Eco-system:

    langchain-core: contains core abstractions involving LangChain Runnables, tooling for observability, and base implementations of important abstractions (e.g., Chat Models).
    langchain: contains generic code that is built using interfaces defined in langchain-core. This package is for code that generalizes well across different implementations of specific interfaces. For example, create_tool_calling_agent works across chat models that support tool calling capabilities.
    langchain-community: community maintained 3rd party integrations. Contains integrations based on interfaces defined in langchain-core. Maintained by the LangChain community.
    Partner Packages (e.g., langchain-[partner]): Partner packages are packages dedicated to especially popular integrations (e.g., langchain-openai, langchain-anthropic etc.). The dedicated packages generally benefit from better reliability and support.
    langgraph: Build robust and stateful multi-actor applications with LLMs by modeling steps as edges and nodes in a graph.
    langserve: Deploy LangChain chains as REST APIs.

In the 0.1.0 release, langchain-community was retained as required a dependency of langchain.

This allowed imports of vectorstores, chat models, and other integrations to continue working through langchain rather than forcing users to update all of their imports to langchain-community.

For the 0.2.0 release, we’re removing the dependency of langchain on langchain-community. This is something we’ve been planning to do since the 0.1 release because we believe this is the right package architecture.

Old imports will continue to work as long as langchain-community is installed. These imports will be removed in the 0.4.0 release.

To understand why we think breaking the dependency of langchain on langchain-community is best we should understand what each package is meant to do.

langchain is meant to contain high-level chains and agent architectures. The logic in these should be specified at the level of abstractions like ChatModel and Retriever, and should not be specific to any one integration. This has two main benefits:

    langchain is fairly lightweight. Here is the full list of required dependencies (after the split)

    python = ">=3.8.1,<4.0"
    langchain-core = "^0.2.0"
    langchain-text-splitters = ">=0.0.1,<0.1"
    langsmith = "^0.1.17"
    pydantic = ">=|=========================================================================================|
1,<3"
    SQLAlchemy = ">=1.4,<3"
    requests = "^2"
    PyYAML = ">=5.3"
    numpy = "^1"
    aiohttp = "^3.8.3"
    tenacity = "^8.1.0"
    jsonpatch = "^1.33"

    langchain chains/agents are largely integration-agnostic, which makes it easy to experiment with different integrations and future-proofs your code should there be issues with one specific integration.

There is also a third less tangible benefit which is that being integration-agnostic forces us to find only those very generic abstractions and architectures which generalize well across integrations. Given how general the abilities of the foundational tech are, and how quickly the space is moving, having generic architectures is a good way of future-proofing your applications.

langchain-community is intended to have all integration-specific components that are not yet being maintained in separate langchain-{partner} packages. Today this is still the majority of integrations and a lot of code. This code is primarily contributed by the community, while langchain is largely written by core maintainers. All of these integrations use optional dependencies and conditional imports, which prevents dependency bloat and conflicts but means compatible dependency versions are not made explicit. Given the volume of integrations in langchain-community and the speed at which integrations change, it’s very hard to follow semver versioning, and we currently don’t.

All of which is to say that there’s no large benefits to langchain depending on langchain-community and some obvious downsides: the functionality in langchain should be integration agnostic anyways, langchain-community can’t be properly versioned, and depending on langchain-community increases the vulnerability surface of langchain.

For more context about the reason for the organization please see our blog: https://blog.langchain.dev/langchain-v0-1-0/"""

In [ ]:
def clean_cuda():
    torch.cuda.empty_cache()

# Useful

In [1]:
import os, torch
os.environ['CUDA_VISIBLE_DEVICES'] = str(4)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

model_id = "microsoft/Phi-3.5-MoE-instruct"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, 
        bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    # Tokenizer initialization
    
model = AutoModelForCausalLM.from_pretrained(
    	model_id, 
    	torch_dtype="auto", 
    	trust_remote_code=True,
        quantization_config=bnb_config
	)
model.to(0)
tokenizer = AutoTokenizer.from_pretrained(model_id)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

You shouldn't move a model that is dispatched using accelerate hooks.


In [16]:
system_command = "You are a summarizer."
user_command = text

messages = [{"role": "system", "content": system_command},
		        {"role": "user", "content": user_command}]

generation_args = {
        "use_cache": True,
        "max_new_tokens": 5000,
        #"temperature": 0.7,
        "repetition_penalty": 1.1,
        #"do_sample": True  # Enable sampling to use temperature
    }
pipe = pipeline(
		"text-generation",
		model=model,
		tokenizer=tokenizer,
	)
	    
output = pipe(messages, **generation_args)

In [17]:
print(output[-1]['generated_text'][-1]['content'])

 **Summary of LangChain v0.2**

LangChain v0.2 introduces several enhancements over its predecessor, including improved streaming support, standardized tool calling, and enhanced async support in Python. Key additions include:

- Enhanced streaming via Event Streaming API.
- Uniform tool calling mechanisms.
- Structured output interfaces.
- Simplified creation of RunnableLambdas with `@chain` decorators.
- Insight tools for analyzing runnable workflows and language graphs.
- Integration of response metadata within AIMessages.
- Visualization tools for runnables and language graph applications.
- Cross-platform compatibility for chat message histories.
- Support for over 20+ partner packages in Python.

Future developments focus on expanding LangGraph's capabilities, refining vector stores, improving documentation, and transitioning to Pydantic 2.

Significant changes involve decoupling LangChain from LangChain-Community, leading to a cleaner separation where LangChain-Core provides fun

In [18]:
from IPython.display import Markdown, display

display(Markdown((output[-1]['generated_text'][-1]['content'])))

 **Summary of LangChain v0.2**

LangChain v0.2 introduces several enhancements over its predecessor, including improved streaming support, standardized tool calling, and enhanced async support in Python. Key additions include:

- Enhanced streaming via Event Streaming API.
- Uniform tool calling mechanisms.
- Structured output interfaces.
- Simplified creation of RunnableLambdas with `@chain` decorators.
- Insight tools for analyzing runnable workflows and language graphs.
- Integration of response metadata within AIMessages.
- Visualization tools for runnables and language graph applications.
- Cross-platform compatibility for chat message histories.
- Support for over 20+ partner packages in Python.

Future developments focus on expanding LangGraph's capabilities, refining vector stores, improving documentation, and transitioning to Pydantic 2.

Significant changes involve decoupling LangChain from LangChain-Community, leading to a cleaner separation where LangChain-Core provides fundamental abstractions, LangChain offers generalized code, LangChain-Community houses community-driven integrations, and LangChain-Partners cater to specialized partnerships.

The restructuring ensures that LangChain remains lightweight, adaptable, and free from unnecessary dependencies, promoting easier maintenance and future-proofing against rapid technological advances. Further details and rationale behind the ecosystem's evolution can be found in the LangChain blog post discussing the v0.1 release.

In [2]:
asd

NameError: name 'asd' is not defined

In [1]:
clean_cuda()

NameError: name 'clean_cuda' is not defined